<a href="https://colab.research.google.com/github/tecasistente5/Anti-Spoofing-Models/blob/master/Yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies


## Google Drive

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Imports

In [ ]:
!pip install ipywidgets
!pip install ultralytics
!pip install matplotlib
!pip install opencv-python
!pip install torch
!pip install opencv-python

In [ ]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

# Camera

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)
  print(img.shape)
  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))
  #print("---------------")
  #print(bbox_bytes)
  #print("---------------")
  return bbox_bytes

In [ ]:
from ultralytics import YOLO
def analize(img):
  model = YOLO("/content/drive/MyDrive/Image-Quality/Models/best.pt")
  results = model(img)
  print(results[-1].probs.top1)
  # return True
  return results[-1].probs.top1

In [ ]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
from matplotlib import pyplot as plt
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])
    if img is not None:
      h,w, _ = img.shape
      model.setInputSize([w,h])
      faces = model.infer(img)
      for det in faces if faces is not None else []: # No se crashee cuando se sale del frame
        bbox = det[0:4].astype(np.int32)
        # Aument the bounding box
        padding_x = int(0.2 * bbox[2])
        padding_y = int(0.2 * bbox[3])
        x1 = max(bbox[0] - padding_x, 0)
        y1 = max(bbox[1] - padding_y, 0)
        x2 = min(bbox[0] + bbox[2] + padding_x, w)
        y2 = min(bbox[1] + bbox[3] + padding_y, h)
        roi = img[y1:y2, x1:x2]
        #cv2.imwrite("roi.png",roi)
        #roi = frame[bbox[1] : bbox[1] + bbox[3], bbox[0] : bbox[0] + bbox[2]]
        roi = img[bbox[1] : bbox[1] + bbox[3], bbox[0] : bbox[0] + bbox[2]]
        cv2.imwrite("roi.jpg", roi)
        if roi.size > 0:
            if analize(roi) == 0:
                info = "live"
            else:
                info = "spoof"
        #cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(img, info, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
        im2Display = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        imtemp = im2Display.copy()
        cv2.rectangle(imtemp, (x1, y1), (x2, y2), (255, 0, 0), 2)
        plt.imshow(imtemp)
        plt.show()


# Menú

## Folder Creation

In [ ]:
from enum import Enum

class FolderName(Enum):
  YOLO = 'Yolo'
  TRAIN = 'train'
  TEST = 'test'

def verify_folder(path):
  if not os.path.exists(path):
    os.makedirs(path)
    print("La carpeta ha sido creada en Google Drive")
  else:
      print("La carpeta ya existe en Google Drive.")

def choose_path(value):
    names = ['Yolo', 'train', 'test'] # List of names of the directories
    base_path = '/content/drive/My Drive' # Root of the google drive
    try:
      folder_name = FolderName(value).value # Uses the enum to get the value

      if folder_name == FolderName.YOLO.value:
          return f'{base_path}/{folder_name}' # Since we need to make the Yolo a root directory, we must clasify it apart
      else:
          return f'{base_path}/{FolderName.YOLO.value}/{folder_name}'
    except ValueError:
      print(f"Valor '{value}' no válido.")
      return None # In case there's no value, it will be catched

def create_path(button, value):
    if value == FolderName.YOLO.value:
        path = choose_path(value)
        if path is not None:
            verify_folder(path)
    else:
        path = choose_path(value)
        if path is not None:
            verify_folder(path)

In [ ]:
from IPython.display import display, clear_output
from ipywidgets import widgets, HBox

def back_to_menu(b):
    # Función para limpiar la pantalla y volver al estado inicial, o realizar otra acción
    clear_output(wait=True)

def folder_setup(b):
  """
  Initializes and displays UI buttons for folder setup in a Jupyter notebook environment.

  This function creates four buttons: 'Create Yolo Folder', 'Create Test Folder', 'Create Train Folder', and 'Home'.
  Each button is configured with a specific action to create corresponding folders when clicked, except the 'Home' button,
  which triggers the main_widgets function. The function makes use of ipywidgets for UI elements and is designed to be used
  within Jupyter notebooks for interactive folder management.

  Parameters:
  - b: The callback parameter passed by the button's on_click event, not used in this function.

  Returns:
  None
  """
  clear_output()
  yolo_folder = widgets.Button(
    description='Create Yolo Folder',
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    )
  test_folder = widgets.Button(
    description='Create Test Folder',
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    )
  train_folder = widgets.Button(
    description='Create Train Folder',
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    )
  home_button = widgets.Button(
    description='Home',
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    )
  yolo_folder.on_click(lambda b: create_path(b, FolderName.YOLO.value))
  test_folder.on_click(lambda b: create_path(b, FolderName.TEST.value))
  train_folder.on_click(lambda b: create_path(b, FolderName.TRAIN.value))
  home_button.on_click(main_widgets)
  items = [yolo_folder, test_folder, train_folder, home_button]
  display(HBox(items))

def models_widget_action(b):
  pass


## Main Interface

https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html

In [ ]:
from ipywidgets import widgets
def main_widgets(b):
  clear_output()
  create_folder = widgets.Button(
      description='Create Folder',
      button_style='info', # 'success', 'info', 'warning', 'danger' or ''
      tooltip='Click me',
  )
  models_button = widgets.Button(
      description='Models',
      button_style='success', # 'success', 'info', 'warning', 'danger' or ''
      tooltip='Click me',
  )
  upload_button = widgets.Button(
    description='Upload Dataset',
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
  )
  create_folder.on_click(folder_setup)
  items = [create_folder, upload_button ,models_button]
  display(widgets.HBox(items))

main_widgets("")


# Models and Implementations

## Yunet

In [ ]:
import cv2 as cv
class YuNet:
    def __init__(self, modelPath, inputSize=[320, 320], confThreshold=0.6, nmsThreshold=0.3, topK=5000, backendId=0, targetId=0):
        self._modelPath = modelPath
        self._inputSize = tuple(inputSize) # [w, h]
        self._confThreshold = confThreshold
        self._nmsThreshold = nmsThreshold
        self._topK = topK
        self._backendId = backendId
        self._targetId = targetId

        self._model = cv.FaceDetectorYN.create(
            model=self._modelPath,
            config="",
            input_size=self._inputSize,
            score_threshold=self._confThreshold,
            nms_threshold=self._nmsThreshold,
            top_k=self._topK,
            backend_id=self._backendId,
            target_id=self._targetId)

    @property
    def name(self):
        return self.__class__.__name__

    def setBackendAndTarget(self, backendId, targetId):
        self._backendId = backendId
        self._targetId = targetId
        self._model = cv.FaceDetectorYN.create(
            model=self._modelPath,
            config="",
            input_size=self._inputSize,
            score_threshold=self._confThreshold,
            nms_threshold=self._nmsThreshold,
            top_k=self._topK,
            backend_id=self._backendId,
            target_id=self._targetId)

    def setInputSize(self, input_size):
        self._model.setInputSize(tuple(input_size))

    def infer(self, image):
        # Forward
        faces = self._model.detect(image)
        return faces[1]


In [ ]:
def Yunet_Model():
  model = YuNet(
      modelPath = "/content/drive/MyDrive/Image-Quality/Models/Yunet/face_detection_yunet_2023mar.onnx",
      inputSize=[320, 320],
      confThreshold=0.9,
      nmsThreshold=0.3,
      topK=100,
      backendId=cv2.dnn.DNN_BACKEND_OPENCV,
      targetId=cv2.dnn.DNN_TARGET_CPU,
)



## Yolo Training

### Yolo


In [ ]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.16 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.3/107.7 GB disk)


In [ ]:
from ultralytics import YOLO
from IPython.display import display, Image

In [ ]:
dataset = "/content/drive/MyDrive/YOLOv8-Test/Dataset/Live-Spoof"

### Model training

In [ ]:
!yolo task=classify mode=train model=yolov8n-cls.pt data="{dataset}" epochs=50 imgsz=128

### Model Validation

In [ ]:
!yolo task=classify mode=val model=/content/runs/classify/train/weights/best.pt data="{dataset}"

In [ ]:
Image(filename=f'/content/runs/classify/val/confusion_matrix.png')


In [ ]:
Image(filename=f'/content/runs/classify/val/val_batch0_labels.jpg')

In [ ]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))
  print(bbpx_bytes)
  return bbox_bytes

In [ ]:
# initialize the Haar Cascade face detection model
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

In [ ]:
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)

  # get photo data
  data = eval_js('takePhoto({})'.format(quality))
  # get OpenCV format image
  img = js_to_image(data)
  #roi =
  # grayscale img
  gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
  print(gray.shape)
  # get face bounding box coordinates using Haar Cascade
  faces = face_cascade.detectMultiScale(gray)
  # draw face bounding box on image
  for (x,y,w,h) in faces:
      img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
  # save image
  cv2.imwrite(filename, img)

  return filename

In [ ]:
try:
  filename = take_photo('photo.jpg')
  print('Saved to {}'.format(filename))

  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

name 'take_photo' is not defined
